In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import brier_score_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from scipy.stats import uniform,randint
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt


In [2]:
data = pd.read_csv('../data/preprocess_data.csv',sep = ',')

In [3]:
pd.options.display.max_columns = None
data.head(5)

,Tournament,Home Team,Away Team,Match Date,Winner Team,Winner Home Or Away,Phase,Final Score,Extra Periods,Team,Opponent Team,Team Points,Team Two Pointers Made,Team Two Pointers Attempted,Team Two Pointers Percenage,Team Three Pointers Made,Team Three Pointers Attempted,Team Three Pointers Percenage,Team Free Throws Made,Team Free Throws Attempted,Team Free Throws Percenage,Team Offensive Rebounds,Team Defensive Rebounds,Team Total Rebounds,Team Assists,Team Steals,Team Turnovers,Team Blocks,Team Blocks Against,Team Personal Fouls,Team Fouls Received,Team Performance Index,Opponent Points,Opponent Two Pointers Made,Opponent Two Pointers Attempted,Opponent Two Pointers Percenage,Opponent Three Pointers Made,Opponent Three Pointers Attempted,Opponent Three Pointers Percenage,Opponent Free Throws Made,Opponent Free Throws Attempted,Opponent Free Throws Percenage,Opponent Offensive Rebounds,Opponent Defensive Rebounds,Opponent Total Rebounds,Opponent Assists,Opponent Steals,Opponent Turnovers,Opponent Blocks,Opponent Blocks Against,Opponent Personal Fouls,Opponent Fouls Received,Opponent Performance Index,Team Result,teamFGA,teamFGM,teamTREB%,teamASST%,teamTS%,teamEFG%,teamOREB%,teamDREB%,teamTO%,teamPoss,teamSTL%,teamBLK%,teamBLKR,teamPPS,teamPlay%,teamAR,teamAST/TO,teamSTL/TO,teamFIC,teamOrtg,opptFGA,opptPoss,teamDrtg,teamEDiff,team_Game_Score,opptFGM,opptTREB%,opptASST%,opptTS%,opptEFG%,opptOREB%,opptDREB%,opptTO%,opptSTL%,opptBLK%,opptBLKR,opptPPS,opptFIC,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,oppt_Game_Score,Points difference,Finals,Last 32,Playoffs,Quarter-Finals,Regular Season,Semifinals,Top 16
0,Basket League,KAOD,Aris,2013-10-12,Aris,Away,Regular Season,58-66,0.0,KAOD,Aris,58.0,20.0,43.0,0.46512,1.0,10.0,0.10000,15.0,26.0,0.57692,12.0,22.0,34.0,6.0,6.0,17.0,3.0,3.0,22.0,22.0,40.0,66.0,15.0,33.0,0.45455,9.0,23.0,0.39130,9.0,19.0,0.47368,10.0,18.0,28.0,11.0,8.0,14.0,3.0,3.0,22.0,22.0,57.0,0.0,53.0,21.0,54.838710,28.571429,45.003104,40.566038,40.000000,68.750000,20.874263,69.44,8.640553,4.320276,9.090909,1.094340,36.206897,9.293680,0.352941,0.352941,24.000,83.525346,56.0,68.36,96.547689,-13.022343,26.4,24.0,45.161290,45.833333,51.274083,50.892857,31.250000,60.000000,17.866258,11.702750,4.388531,6.976744,1.178571,37.375,96.547689,83.525346,13.022343,40.000000,15.143172,0.785714,0.571429,39.8,-8.0,0,0,0,0,1,0,0
1,Basket League,PAOK,Kolossos Rhodes,2013-10-12,PAOK,Home,Regular Season,88-81,0.0,PAOK,Kolossos Rhodes,88.0,17.0,35.0,0.48571,10.0,26.0,0.38462,24.0,26.0,0.92308,9.0,22.0,31.0,22.0,7.0,18.0,3.0,1.0,23.0,23.0,96.0,81.0,26.0,42.0,0.61905,5.0,16.0,0.31250,14.0,21.0,0.66667,6.0,25.0,31.0,5.0,8.0,18.0,1.0,3.0,24.0,22.0,69.0,1.0,61.0,27.0,50.000000,81.481481,60.739923,52.459016,26.470588,78.571429,19.902698,81.44,8.595285,3.683694,7.142857,1.442623,38.571429,24.564538,1.222222,0.388889,60.500,108.055010,58.0,79.24,102.221100,5.833909,65.5,31.0,50.000000,16.129032,60.232005,57.758621,21.428571,73.529412,21.116847,10.095911,1.261989,2.857143,1.396552,38.375,102.221100,108.055010,-5.833909,44.285714,6.967670,0.277778,0.444444,46.3,7.0,0,0,0,0,1,0,0
2,Liga ACB,Bilbao,Zaragoza,2013-10-12,Zaragoza,Away,Regular Season,77-86,0.0,Bilbao,Zaragoza,77.0,24.0,46.0,0.52000,5.0,12.0,0.42000,14.0,20.0,0.70000,13.0,19.0,32.0,15.0,7.0,12.0,1.0,1.0,18.0,23.0,89.0,86.0,25.0,39.0,0.64000,10.0,23.0,0.43000,6.0,10.0,0.60000,9.0,20.0,29.0,16.0,10.0,10.0,1.0,1.0,23.0,18.0,95.0,0.0,58.0,29.0,52.459016,51.724138,57.634731,54.310345,39.393939,67.857143,15.228426,65.80,10.638298,1.519757,2.564103,1.327586,50.877193,19.685039,1.250000,0.583333,55.250,117.021277,62.0,67.40,127.596439,-10.575163,59.4,35.0,47.540984,45.714286,64.759036,64.516129,32.142857,60.606061,13.089005,14.836795,1.483680,2.173913,1.387097,65.250,127.596439,117.021277,10.575163,55.555556,19.138756,1.600000,1.000000,70.0,-9.0,0,0,0,0,1,0,0
3,Basket League,Trikala,Kifisia,2013-10-12,Kifisia,Away,Regular Season,64-69,0.0,Trikala,Kifisia,64.0,21.0,41.0,0.51220,

In [4]:
data["Tournament"].unique()

array(['Basket League', 'Liga ACB', 'Eurocup', 'Euroleague'], dtype=object)

# Teams as Dummy Variables

In [5]:
baseline_data = data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]]
baseline_data.head()

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase
0,Basket League,2013-10-12,KAOD,Aris,0.0,Aris,Regular Season
1,Basket League,2013-10-12,PAOK,Kolossos Rhodes,1.0,PAOK,Regular Season
2,Liga ACB,2013-10-12,Bilbao,Zaragoza,0.0,Zaragoza,Regular Season
3,Basket League,2013-10-12,Trikala,Kifisia,0.0,Kifisia,Regular Season
4,Liga ACB,2013-10-12,Tenerife,Obradoiro,1.0,Tenerife,Regular Season


## Greek Basket League 

In [6]:
greek_baseline_data = baseline_data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]][baseline_data["Tournament"]=='Basket League'].reset_index(drop = True)
greek_baseline_data.head()

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase
0,Basket League,2013-10-12,KAOD,Aris,0.0,Aris,Regular Season
1,Basket League,2013-10-12,PAOK,Kolossos Rhodes,1.0,PAOK,Regular Season
2,Basket League,2013-10-12,Trikala,Kifisia,0.0,Kifisia,Regular Season
3,Basket League,2013-10-13,Panelefsiniakos,Panionios,0.0,Panionios,Regular Season
4,Basket League,2013-10-13,Rethymno,Ikaros,1.0,Rethymno,Regular Season


In [7]:
greece_home = pd.get_dummies(greek_baseline_data["Home Team"]).reset_index(drop = True)
greece_away = -1*pd.get_dummies(greek_baseline_data["Away Team"]).reset_index(drop = True)
greece_x = pd.concat([greece_home,greece_away],axis=1).groupby(level=0, axis=1).sum()
greece_y = greek_baseline_data["Team Result"]

In [8]:
greek_baseline_data['Phase'].unique()

array(['Regular Season', 'Playoffs'], dtype=object)

In [9]:
x_structure_greece = greece_x.iloc[:len(greek_baseline_data[greek_baseline_data["Match Date"]<"2017-08-01"])]
y_structure_greece = greece_y.iloc[:len(greek_baseline_data[greek_baseline_data["Match Date"]<"2017-08-01"])]

start = int(len(greek_baseline_data[greek_baseline_data["Match Date"]<"2017-08-01"]))
end = start + int(len(greek_baseline_data[(greek_baseline_data["Match Date"]>"2017-08-01") & (greek_baseline_data['Phase'].isin(['Regular Season']))]))

x_train_greece = greece_x.iloc[start:end]
y_train_greece = greece_y.iloc[start:end]


x_test_greece = greece_x.iloc[end:]
y_test_greece = greece_y.iloc[end:]

###  Logistic Regression

In [10]:
lgr = LogisticRegression(C = 0.3839820885947254,penalty = 'l1', solver= 'liblinear').fit(x_train_greece, y_train_greece)
y_pre_proba_lgr=lgr.predict_proba(x_test_greece)
print("Logistic Regression")
print()
print("Test BS:",round(brier_score_loss(y_test_greece,y_pre_proba_lgr[:,1]),3))
print()
y_pre_lgr=np.where(pd.Series(y_pre_proba_lgr[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_lgr,y_test_greece),3))
print()
print("Test F1-score:",round(f1_score(y_pre_lgr,y_test_greece),3))


Logistic Regression

Test BS: 0.16

Test Accuracy: 0.727

Test F1-score 0.786


In [11]:
coef_dict = {}
for coef, feat in zip(lgr.coef_[:].tolist()[0],greece_x.columns.values):
    coef_dict[feat] = coef
coef1=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef1=coef1.iloc[(-coef1[0].abs()).argsort()].reset_index()

In [12]:
sorted_coef1

,index,0
0,Panathinaikos,2.449945
1,Olympiacos,1.328191
2,Trikala,-1.057064
3,Koroivos,-0.479360
4,Panionios,-0.479358
5,PAOK,0.362588
6,Promitheas Patras,0.362583
7,Ifaistos Limnou,-0.300507
8,AEK,0.012990
9,Kavala,0.000000


In [13]:
lgr.intercept_

array([0.65623536])

## Random Forest Classifier

In [16]:
rfc = RandomForestClassifier(max_depth= 38, max_features='sqrt', min_samples_leaf=0.0022712801155358453,
                             min_samples_split=0.009889030194338, n_estimators=53
                             ,random_state= 1).fit(x_train_greece, y_train_greece)

y_pre_proba_rfc=rfc.predict_proba(x_test_greece)
print("Random Forest Classifier")
print()
print("Test BS:",round(brier_score_loss(y_test_greece,y_pre_proba_rfc[:,1]),3))
print()
y_pre_rfc=np.where(pd.Series(y_pre_proba_rfc[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_rfc,y_test_greece),3))
print()
print("Test F1-score:",round(f1_score(y_pre_rfc,y_test_greece),3))

Random Forest Classifier

Test BS: 0.197

Test Accuracy: 0.727

Test F1-score 0.769


In [17]:
coef_dict = {}
for coef, feat in zip(rfc.feature_importances_[:].tolist(),greece_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()
sorted_coef2

,index,0
0,Panathinaikos,0.167779
1,Trikala,0.100447
2,Olympiacos,0.094208
3,Koroivos,0.071499
4,PAOK,0.068532
5,Promitheas Patras,0.061040
6,Aris,0.060697
7,Kolossos Rhodes,0.060263
8,Lavrio,0.057169
9,Ifaistos Limnou,0.056117


## xgboost

In [18]:
alg = XGBClassifier(colsample_bytree = 0.7120387144249468, gamma = 1.173482282705095, learning_rate = 0.18114216198386113,
                    max_depth= 41, n_estimators= 88, reg_lambda= 0.33962213881868086, subsample= 0.16627201904949962, 
                    objective='binary:logistic').fit(x_train_greece, y_train_greece)




y_pre_proba_alg=alg.predict_proba(x_test_greece)
print("xgboost")
print()
print("Test BS:",round(brier_score_loss(y_test_greece,y_pre_proba_alg[:,1]),3))
print()
y_pre_alg=np.where(pd.Series(y_pre_proba_alg[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_alg,y_test_greece),3))
print()
print("Test F1-score:",round(f1_score(y_pre_alg,y_test_greece),3))


xgboost

Test BS: 0.202

Test Accuracy: 0.636

Test F1-score 0.75


In [19]:
coef_dict = {}
for coef, feat in zip(alg.feature_importances_[:].tolist(),greece_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()
sorted_coef3

,index,0
0,AEK,0.231502
1,Olympiacos,0.166736
2,Panionios,0.136841
3,Panathinaikos,0.125185
4,Ifaistos Limnou,0.082923
5,Trikala,0.082287
6,Promitheas Patras,0.064689
7,PAOK,0.058332
8,Lavrio,0.051504
9,Apollon Patras,0.000000


# Ensemble

In [20]:
ens_proba=(y_pre_proba_lgr[:,1]+y_pre_proba_rfc[:,1]+y_pre_proba_alg[:,1])/3


print("------Ensemble 3 classifier-------")
print()
print("Test BS:",round(brier_score_loss(y_test_greece,ens_proba),3))
print()
y_pre_ens=np.where(pd.Series(ens_proba)>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_ens,y_test_greece),3))
print()
print("Test F1-score:",round(f1_score(y_pre_ens,y_test_greece),3))


------Ensemble 3 classifier-------

Test BS: 0.169

Test Accuracy: 0.773

Test F1-score 0.815


In [21]:
matches = greek_baseline_data[(greek_baseline_data["Match Date"]>"2017-08-01") & (greek_baseline_data['Phase'].isin(['Playoffs']))]
matches['Probability']=ens_proba
matches['Prediction']=np.where(matches['Probability']>0.5,1,0)
matches['BS']= (matches['Team Result'] -  matches['Probability'])**2
matches

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase,Probability,Prediction,BS
1007,Basket League,2018-05-16,PAOK,AEK,1.0,PAOK,Playoffs,0.791983,1,0.043271
1008,Basket League,2018-05-16,Panathinaikos,Kolossos Rhodes,1.0,Panathinaikos,Playoffs,0.865407,1,0.018115
1009,Basket League,2018-05-16,Promitheas Patras,Lavrio,1.0,Promitheas Patras,Playoffs,0.785057,1,0.046200
1010,Basket League,2018-05-16,Olympiacos,Kymi,1.0,Olympiacos,Playoffs,0.862221,1,0.018983
1011,Basket League,2018-05-19,Kolossos Rhodes,Panathinaikos,0.0,Panathinaikos,Playoffs,0.164832,0,0.027170
1012,Basket League,2018-05-19,Lavrio,Promitheas Patras,0.0,Promitheas Patras,Playoffs,0.670466,1,0.449525
1013,Basket League,2018-05-19,Kymi,Olympiacos,0.0,Olympiacos,Playoffs,0.329515,0,0.108580
1014,Basket League,2018-05-19,AEK,PAOK,0.0,PAOK,Playoffs,0.460542,0,0.212099
1015,Basket League,2018-05-24,Olympiacos,Promitheas Patras,1.0,Olympiacos,Playoffs,0.787695,1,0.045074
1016,Basket League,2018-05-24,Panathinaikos,PAOK,1.0,Panathinaikos,Playoffs,0.858048,1,0.020150


# Liga ACB

In [22]:
spain_baseline_data = baseline_data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]][baseline_data["Tournament"]=='Liga ACB'].reset_index(drop = True)
spain_baseline_data.head()

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase
0,Liga ACB,2013-10-12,Bilbao,Zaragoza,0.0,Zaragoza,Regular Season
1,Liga ACB,2013-10-12,Tenerife,Obradoiro,1.0,Tenerife,Regular Season
2,Liga ACB,2013-10-13,Manresa,Joventut,1.0,Manresa,Regular Season
3,Liga ACB,2013-10-13,Baskonia,Gran Canaria,0.0,Gran Canaria,Regular Season
4,Liga ACB,2013-10-13,Gipuzkoa,Real Betis,1.0,Gipuzkoa,Regular Season


In [23]:
spain_home = pd.get_dummies(spain_baseline_data["Home Team"]).reset_index(drop = True)
spain_away = -1*pd.get_dummies(spain_baseline_data["Away Team"]).reset_index(drop = True)
spain_x = pd.concat([spain_home,spain_away],axis=1).groupby(level=0, axis=1).sum()
spain_y = spain_baseline_data["Team Result"]

In [24]:
spain_x.head()

,Andorra,Barcelona,Baskonia,Bilbao,Burgos,Estudiantes,Fuenlabrada,Gipuzkoa,Gran Canaria,Joventut,Manresa,Murcia,Obradoiro,Real Betis,Real Madrid,Tenerife,Unicaja Malaga,Valencia,Valladolid,Zaragoza
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1
1,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,-1,0,0,0,0,0,0


In [25]:
spain_baseline_data['Phase'].unique()

array(['Regular Season', 'Playoffs'], dtype=object)

In [26]:
x_structure_spain = spain_x.iloc[:len(spain_baseline_data[spain_baseline_data["Match Date"]<"2017-08-01"])]
y_structure_spain = spain_y.iloc[:len(spain_baseline_data[spain_baseline_data["Match Date"]<"2017-08-01"])]

start = int(len(spain_baseline_data[spain_baseline_data["Match Date"]<"2017-08-01"]))
end = start + int(len(spain_baseline_data[(spain_baseline_data["Match Date"]>"2017-08-01") & (spain_baseline_data['Phase'].isin(['Regular Season']))]))

x_train_spain = spain_x.iloc[start:end]
y_train_spain = spain_y.iloc[start:end]


x_test_spain = spain_x.iloc[end:]
y_test_spain = spain_y.iloc[end:]

###  Logistic Regression

In [75]:
lgr = LogisticRegression(solver='liblinear',C = 0.6765129921671309, penalty = 'l1').fit(x_train_spain, y_train_spain)
y_pre_proba_lgr=lgr.predict_proba(x_test_spain)
print("Logistic Regression")
print()
print("Test BS:",round(brier_score_loss(y_test_spain,y_pre_proba_lgr[:,1]),3))
print()
y_pre_lgr=np.where(pd.Series(y_pre_proba_lgr[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_lgr,y_test_spain),3))
print()
print("Test F1-score:",round(f1_score(y_pre_lgr,y_test_spain),3))


Logistic Regression

Test BS: 0.195

Test Accuracy: 0.714

Test F1-score 0.786


In [76]:
coef_dict = {}
for coef, feat in zip(lgr.coef_[:].tolist()[0],spain_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()

In [77]:
sorted_coef2

,index,0
0,Real Madrid,0.930548
1,Real Betis,-0.497974
2,Bilbao,-0.375523
3,Baskonia,0.294738
4,Barcelona,0.176243
5,Zaragoza,-0.137166
6,Valencia,0.000000
7,Unicaja Malaga,0.000000
8,Tenerife,0.000000
9,Obradoiro,0.000000


In [78]:
lgr.intercept_

array([0.38859978])

## Random Forest Classifier

In [79]:
rfc = RandomForestClassifier(max_depth= 39, max_features='sqrt', min_samples_leaf=0.0009330471732301171,
                             min_samples_split=0.075312968832076, n_estimators=73,
                             random_state=1).fit(x_train_spain, y_train_spain)

y_pre_proba_rfc=rfc.predict_proba(x_test_spain)
print("Random Forest Classifier")
print()
print("Test BS:",round(brier_score_loss(y_test_spain,y_pre_proba_rfc[:,1]),3))
print()
y_pre_rfc=np.where(pd.Series(y_pre_proba_rfc[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_rfc,y_test_spain),3))
print()
print("Test F1-score:",round(f1_score(y_pre_rfc,y_test_spain),3))

Random Forest Classifier

Test BS: 0.221

Test Accuracy: 0.714

Test F1-score 0.786


In [80]:
coef_dict = {}
for coef, feat in zip(rfc.feature_importances_[:].tolist(),spain_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()
sorted_coef2

,index,0
0,Real Madrid,0.076281
1,Real Betis,0.070830
2,Barcelona,0.064067
3,Zaragoza,0.060266
4,Murcia,0.059403
5,Andorra,0.057632
6,Unicaja Malaga,0.056542
7,Fuenlabrada,0.054448
8,Obradoiro,0.053696
9,Bilbao,0.053330


## xgboost

In [81]:
alg = XGBClassifier(colsample_bytree= 0.6188949028768943, gamma= 2.819830261508139, learning_rate= 0.36256801131990146,
                    max_depth= 21, n_estimators= 86, reg_lambda= 0.09211406347716067, subsample= 0.5079441274104476,
                    objective='binary:logistic').fit(x_train_spain, y_train_spain)

y_pre_proba_alg=alg.predict_proba(x_test_spain)
print("xgboost")
print()
print("Test BS:",round(brier_score_loss(y_test_spain,y_pre_proba_alg[:,1]),3))
print()
y_pre_alg=np.where(pd.Series(y_pre_proba_alg[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_alg,y_test_spain),3))
print()
print("Test F1-score:",round(f1_score(y_pre_alg,y_test_spain),3))


xgboost

Test BS: 0.194

Test Accuracy: 0.714

Test F1-score 0.769


In [82]:
coef_dict = {}
for coef, feat in zip(alg.feature_importances_[:].tolist(),spain_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()
sorted_coef3

,index,0
0,Real Madrid,0.076847
1,Real Betis,0.075006
2,Zaragoza,0.056869
3,Joventut,0.056765
4,Baskonia,0.056744
5,Gran Canaria,0.056598
6,Murcia,0.055826
7,Obradoiro,0.055069
8,Gipuzkoa,0.055058
9,Burgos,0.053649


In [83]:
ens_proba=(y_pre_proba_lgr[:,1]+y_pre_proba_rfc[:,1]+y_pre_proba_alg[:,1])/3


print("------Ensemble 3 classifier-------")
print()
print("Test BS:",round(brier_score_loss(y_test_spain,ens_proba),3))
print()
y_pre_ens=np.where(pd.Series(ens_proba)>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_ens,y_test_spain),3))
print()
print("Test F1-score:",round(f1_score(y_pre_ens,y_test_spain),3))


------Ensemble 3 classifier-------

Test BS: 0.19

Test Accuracy: 0.714

Test F1-score 0.786


In [84]:
matches = spain_baseline_data[(spain_baseline_data["Match Date"]>"2017-08-01") & (spain_baseline_data['Phase'].isin(['Playoffs']))]
matches['Probability']=y_pre_proba_lgr[:,1]
matches['Prediction']=np.where(matches['Probability']>0.5,1,0)
matches['BS']= (matches['Team Result'] -  matches['Probability'])**2
matches

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase,Probability,Prediction,BS
1586,Liga ACB,2018-05-27,Barcelona,Andorra,0.0,Andorra,Playoffs,0.637572,1,0.406499
1587,Liga ACB,2018-05-27,Baskonia,Unicaja Malaga,1.0,Baskonia,Playoffs,0.664483,1,0.112571
1588,Liga ACB,2018-05-27,Real Madrid,Tenerife,1.0,Real Madrid,Playoffs,0.789040,1,0.044504
1589,Liga ACB,2018-05-28,Valencia,Gran Canaria,1.0,Valencia,Playoffs,0.595946,1,0.163260
1590,Liga ACB,2018-05-29,Tenerife,Real Madrid,0.0,Real Madrid,Playoffs,0.367734,0,0.135229
1591,Liga ACB,2018-05-29,Unicaja Malaga,Baskonia,0.0,Baskonia,Playoffs,0.523448,1,0.273998
1592,Liga ACB,2018-05-30,Andorra,Barcelona,0.0,Barcelona,Playoffs,0.552891,1,0.305688
1593,Liga ACB,2018-05-30,Gran Canaria,Valencia,1.0,Gran Canaria,Playoffs,0.595946,1,0.163260
1594,Liga ACB,2018-06-01,Valencia,Gran Canaria,0.0,Gran Canaria,Playoffs,0.595946,1,0.355151
1595,Liga ACB,2018-06-01,Barcelona,Andorra,1.0,Barcelona,Playoffs,0.637572,1,0.131354


In [39]:
np.mean(matches['BS'])

0.17982787032709946

# Euroleague

In [40]:
el_baseline_data = baseline_data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]][baseline_data["Tournament"]=='Euroleague'].reset_index(drop = True)
el_baseline_data.head()

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase
0,Euroleague,2013-10-16,Bamberg,Strasbourg,1.0,Bamberg,Regular Season
1,Euroleague,2013-10-16,Siena,Galatasaray,0.0,Galatasaray,Regular Season
2,Euroleague,2013-10-17,Barcelona,Partizan,1.0,Barcelona,Regular Season
3,Euroleague,2013-10-17,Nanterre,CSKA Moscow,0.0,CSKA Moscow,Regular Season
4,Euroleague,2013-10-17,Anadolu Efes,Armani Milano,1.0,Anadolu Efes,Regular Season


In [41]:
el_home = pd.get_dummies(el_baseline_data["Home Team"]).reset_index(drop = True)
el_away = -1*pd.get_dummies(el_baseline_data["Away Team"]).reset_index(drop = True)
el_x = pd.concat([el_home,el_away],axis=1).groupby(level=0, axis=1).sum()
el_y = el_baseline_data["Team Result"]

In [42]:
el_x.head()

,Alba Berlin,Anadolu Efes,Armani Milano,Bamberg,Barcelona,Baskonia,Bayern Munich,Budivelnyk,CSKA Moscow,Cedevita,Crvena Zvezda,Darussafaka,Dinamo Sassari,Fenerbahce,Galatasaray,Khimki,Limoges,Lokomotiv Kuban,Maccabi Tel Aviv,Nanterre,Neptunas,Nizhny Novgorod,Olympiacos,Panathinaikos,Partizan,Pinar Karsiyaka,Real Madrid,Rytas Vilnius,Siena,Strasbourg,Turow Zgorzelec,Unicaja Malaga,Unics Kazan,Valencia,Zalgiris,Zielona Gora
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
el_baseline_data['Phase'].unique()

array(['Regular Season', 'Top 16', 'Playoffs', 'Final Four'], dtype=object)

In [44]:
x_structure_el = el_x.iloc[:len(el_baseline_data[el_baseline_data["Match Date"]<"2017-08-01"])]
y_structure_el = el_y.iloc[:len(el_baseline_data[el_baseline_data["Match Date"]<"2017-08-01"])]

start = int(len(el_baseline_data[el_baseline_data["Match Date"]<"2017-08-01"]))
end = start + int(len(el_baseline_data[(el_baseline_data["Match Date"]>"2017-08-01") & (el_baseline_data['Phase'].isin(['Regular Season', 'Top 16']))]))

x_train_el = el_x.iloc[start:end]
y_train_el = el_y.iloc[start:end]


x_test_el = el_x.iloc[end:]
y_test_el = el_y.iloc[end:]

###  Logistic Regression

In [45]:
lgr = LogisticRegression(solver='liblinear' , C= 0.7847137962592965, penalty='l1').fit(x_train_el, y_train_el)
y_pre_proba_lgr=lgr.predict_proba(x_test_el)
print("Logistic Regression")
print()
print("Test BS:",round(brier_score_loss(y_test_el,y_pre_proba_lgr[:,1]),3))
print()
y_pre_lgr=np.where(pd.Series(y_pre_proba_lgr[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_lgr,y_test_el),3))
print()
print("Test F1-score:",round(f1_score(y_pre_lgr,y_test_el),3))


Logistic Regression

Test BS: 0.221

Test Accuracy: 0.7

Test F1-score 0.8


In [46]:
coef_dict = {}
for coef, feat in zip(lgr.coef_[:].tolist()[0],el_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()

In [47]:
sorted_coef3.head(50)

,index,0
0,CSKA Moscow,1.227358
1,Anadolu Efes,-0.972115
2,Fenerbahce,0.729430
3,Armani Milano,-0.499163
4,Panathinaikos,0.429815
5,Olympiacos,0.429810
6,Real Madrid,0.429790
7,Crvena Zvezda,-0.351796
8,Barcelona,-0.351791
9,Bamberg,-0.351767


In [48]:
lgr.intercept_

array([0.5697071])

## Random Forest Classifier

In [49]:
rfc = RandomForestClassifier(max_depth= 36, max_features='log2', min_samples_leaf=0.002390948900012013,
                             min_samples_split=0.0096605683266830, n_estimators=73,
                             random_state=1).fit(x_train_el, y_train_el)

y_pre_proba_rfc=rfc.predict_proba(x_test_el)
print("Random Forest Classifier")
print()
print("Test BS:",round(brier_score_loss(y_test_el,y_pre_proba_rfc[:,1]),3))
print()
y_pre_rfc=np.where(y_pre_proba_rfc[:,1] > 0.5, 1, 0)
print("Test Accuracy:",round(accuracy_score(y_pre_rfc ,y_test_el),3))
print()
print("Test F1-score:",round(f1_score(y_pre_rfc,y_test_el),3))

Random Forest Classifier

Test BS: 0.283

Test Accuracy: 0.6

Test F1-score 0.692


In [50]:
coef_dict = {}
for coef, feat in zip(rfc.feature_importances_[:].tolist(),el_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()
sorted_coef2

,index,0
0,Fenerbahce,0.078747
1,Armani Milano,0.075279
2,CSKA Moscow,0.075127
3,Anadolu Efes,0.073023
4,Olympiacos,0.065112
5,Unicaja Malaga,0.064515
6,Bamberg,0.063880
7,Baskonia,0.059415
8,Maccabi Tel Aviv,0.058967
9,Barcelona,0.058415


## xgboost

In [51]:
alg = XGBClassifier(colsample_bytree= 0.897167919370632, gamma= 0.835671500072126, learning_rate= 0.3887154858505634,
                    max_depth= 1, n_estimators= 99, reg_lambda= 0.7909237442174416, subsample= 0.963659261900305,
                    objective='binary:logistic').fit(x_train_el, y_train_el)

y_pre_proba_alg=alg.predict_proba(x_test_el)
print("xgboost")
print()
print("Test BS:",round(brier_score_loss(y_test_el,y_pre_proba_alg[:,1]),3))
print()
y_pre_alg=np.where(y_pre_proba_alg[:,1] > 0.5, 1, 0)
print("Test Accuracy:",round(accuracy_score(y_pre_alg ,y_test_el),3))
print()
print("Test F1-score:",round(f1_score(y_pre_alg,y_test_el),3))


xgboost

Test BS: 0.26

Test Accuracy: 0.65

Test F1-score 0.759


In [52]:
coef_dict = {}
for coef, feat in zip(alg.feature_importances_[:].tolist(),el_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()
sorted_coef3

,index,0
0,Fenerbahce,0.132523
1,Anadolu Efes,0.106980
2,Armani Milano,0.106158
3,CSKA Moscow,0.080699
4,Khimki,0.063594
5,Panathinaikos,0.061648
6,Unicaja Malaga,0.059631
7,Real Madrid,0.056615
8,Bamberg,0.054916
9,Valencia,0.051035


In [53]:
ens_proba=(y_pre_proba_lgr[:,1]+y_pre_proba_rfc[:,1]+y_pre_proba_alg[:,1])/3


print("------Ensemble 3 classifier-------")
print()
print("Test RPS:",round(brier_score_loss(y_test_el,ens_proba),3))
print()
y_pre_ens=np.where(pd.Series(ens_proba)>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_ens,y_test_el),3))
print()
print("Test F1-score:",round(f1_score(y_pre_ens,y_test_el),3))


------Ensemble 3 classifier-------

Test RPS: 0.246

Test Accuracy: 0.65

Test F1-score 0.759


# Eurocup

In [54]:
ec_baseline_data = baseline_data[["Tournament","Match Date","Home Team","Away Team","Team Result","Winner Team","Phase"]][baseline_data["Tournament"]=='Eurocup'].reset_index(drop = True)
ec_baseline_data.head()

,Tournament,Match Date,Home Team,Away Team,Team Result,Winner Team,Phase
0,Eurocup,2013-10-15,Bonn,Alba Berlin,0.0,Alba Berlin,Regular Season
1,Eurocup,2013-10-15,Banvit,Kalev Cramo,1.0,Banvit,Regular Season
2,Eurocup,2013-10-15,PAOK,Buducnost,1.0,PAOK,Regular Season
3,Eurocup,2013-10-15,Bilbao,Cedevita,1.0,Bilbao,Regular Season
4,Eurocup,2013-10-15,Paris Levallois,Valencia,0.0,Valencia,Regular Season


In [55]:
ec_home = pd.get_dummies(ec_baseline_data["Home Team"]).reset_index(drop = True)
ec_away = -1*pd.get_dummies(ec_baseline_data["Away Team"]).reset_index(drop = True)
ec_x = pd.concat([ec_home,ec_away],axis=1).groupby(level=0, axis=1).sum()
ec_y = ec_baseline_data["Team Result"]

In [56]:
ec_x.head()

,AEK,ASVEL Villeurbanne,Alba Berlin,Alba Fehervar,Andorra,Aris,Armani Milano,Artland Dragons,Avtodor Saratov,Bamberg,Banvit,Bayern Munich,Besiktas,Bilbao,Bisons Loimaa,Bonn,Brindisi,Budivelnyk,Buducnost,Cantu,Cedevita,Chalon Saone,Cibona,Crvena Zvezda,Csu Asesoft Ploiesti,Darussafaka,Dijon,Dinamo Sassari,Fuenlabrada,Galatasaray,Gran Canaria,Gravelines,Hapoel Jerusalem,Igokea,Kalev Cramo,Khimik,Khimki,Krasny Oktyabr,Le Mans,Lietkabelis,Limoges,Lokomotiv Kuban,Ludwigsburg,Lukoil Academic,MZT Skopje,Maccabi Haifa,Maccabi Tel Aviv,Mons,Murcia,Nancy,Nanterre,Neptunas,Nizhny Novgorod,Nymburk,Oldenburg,Olimpija Ljubljana,Oostende,PAOK,Panionios,Paris Levallois,Partizan,Pinar Karsiyaka,Radnicki,Real Betis,Reggio Emilia,Rytas Vilnius,Siena,Spartak St Petersburg,Spirou Charleroi,Steaua Bucharest,Strasbourg,Szolnoki,Ted Ankara Kolejliler,Tofas Bursa,Trabzonspor,Trento,Turin,Turow Zgorzelec,Ulm,Unicaja Malaga,Unics Kazan,VEF Riga,Valencia,Varese,Venezia,Ventspils,Virtus Roma,Zaragoza,Zenit St Petersburg,Zielona Gora
0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0


In [57]:
ec_baseline_data['Phase'].unique()

array(['Regular Season', 'Last 32', 'Quarter-Finals', 'Semifinals',
       'Finals', 'Top 16'], dtype=object)

In [58]:
x_structure_ec = ec_x.iloc[:len(ec_baseline_data[ec_baseline_data["Match Date"]<"2017-08-01"])]
y_structure_ec = ec_y.iloc[:len(ec_baseline_data[ec_baseline_data["Match Date"]<"2017-08-01"])]

start = int(len(ec_baseline_data[ec_baseline_data["Match Date"]<"2017-08-01"]))
end = start + int(len(ec_baseline_data[(ec_baseline_data["Match Date"]>"2017-08-01") & (ec_baseline_data['Phase'].isin(['Regular Season','Last 32', 'Top 16']))]))

x_train_ec = ec_x.iloc[start:end]
y_train_ec = ec_y.iloc[start:end]


x_test_ec = ec_x.iloc[end:]
y_test_ec = ec_y.iloc[end:]

###  Logistic Regression

In [59]:
lgr = LogisticRegression(solver='liblinear',C= 0.8479198446641489, penalty= 'l1').fit(x_train_ec, y_train_ec)
y_pre_proba_lgr=lgr.predict_proba(x_test_ec)
print("Logistic Regression")
print()
print("Test BS:",round(brier_score_loss(y_test_ec,y_pre_proba_lgr[:,1]),3))
print()
y_pre_lgr=np.where(pd.Series(y_pre_proba_lgr[:,1])>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_lgr,y_test_ec),3))
print()
print("Test F1-score:",round(f1_score(y_pre_lgr,y_test_ec),3))


Logistic Regression

Test BS: 0.206

Test Accuracy: 0.688

Test F1-score 0.762


In [60]:
coef_dict = {}
for coef, feat in zip(lgr.coef_[:].tolist()[0],ec_x.columns.values):
    coef_dict[feat] = coef
coef4=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef4=coef4.iloc[(-coef4[0].abs()).argsort()].reset_index()

In [61]:
sorted_coef4.head(50)

,index,0
0,Lokomotiv Kuban,2.424908
1,Bayern Munich,1.514385
2,Partizan,-1.330111
3,Darussafaka,1.062865
4,Bilbao,-0.863033
5,Ulm,-0.654531
6,Paris Levallois,-0.579100
7,Unics Kazan,0.555831
8,Hapoel Jerusalem,-0.447778
9,Zenit St Petersburg,0.363036


In [62]:
lgr.intercept_

array([0.40398155])

## Random Forest Classifier

In [63]:
rfc = RandomForestClassifier(max_depth= 38, max_features='sqrt', min_samples_leaf=0.0037012163422379674,
                             min_samples_split=0.251132570859313, n_estimators=41,
                             random_state=1).fit(x_train_ec, y_train_ec)

y_pre_proba_rfc=rfc.predict_proba(x_test_ec)
print("Random Forest Classifier")
print()
print("Test BS:",round(brier_score_loss(y_test_ec,y_pre_proba_rfc[:,1]),3))
print()
y_pre_rfc=np.where(y_pre_proba_rfc[:,1] > 0.5, 1, 0)
print("Test Accuracy:",round(accuracy_score(y_pre_rfc ,y_test_ec),3))
print()
print("Test F1-score:",round(f1_score(y_pre_rfc,y_test_ec),3))

Random Forest Classifier

Test BS: 0.168

Test Accuracy: 0.75

Test F1-score 0.833


In [64]:
coef_dict = {}
for coef, feat in zip(rfc.feature_importances_[:].tolist(),ec_x.columns.values):
    coef_dict[feat] = coef
coef2=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef2=coef2.iloc[(-coef2[0].abs()).argsort()].reset_index()
sorted_coef2

,index,0
0,Lokomotiv Kuban,0.137547
1,Partizan,0.089420
2,Bilbao,0.082910
3,Bayern Munich,0.070744
4,Hapoel Jerusalem,0.068471
...,...,...
85,Igokea,0.000000
86,Gravelines,0.000000
87,Fuenlabrada,0.000000
88,Dijon,0.000000


## xgboost

In [65]:
alg = XGBClassifier(colsample_bytree= 0.5913621206231853, gamma= 4.08026456094463, learning_rate= 0.37903160103981465,
                    max_depth= 47, n_estimators= 94, reg_lambda= 0.11419103539605524, subsample= 0.7917278647223511,
                    objective='binary:logistic').fit(x_train_ec, y_train_ec)

y_pre_proba_alg=alg.predict_proba(x_test_ec)
print("xgboost")
print()
print("Test BS:",round(brier_score_loss(y_test_ec,y_pre_proba_alg[:,1]),3))
print()
y_pre_alg=np.where(y_pre_proba_alg[:,1] > 0.5, 1, 0)
print("Test Accuracy:",round(accuracy_score(y_pre_alg ,y_test_ec),3))
print()
print("Test F1-score:",round(f1_score(y_pre_alg,y_test_ec),3))


xgboost

Test BS: 0.213

Test Accuracy: 0.688

Test F1-score 0.762


In [66]:
coef_dict = {}
for coef, feat in zip(alg.feature_importances_[:].tolist(),ec_x.columns.values):
    coef_dict[feat] = coef
coef3=pd.DataFrame.from_dict(coef_dict,orient ='index')
sorted_coef3=coef3.iloc[(-coef3[0].abs()).argsort()].reset_index()
sorted_coef3

,index,0
0,Unics Kazan,0.162376
1,Bilbao,0.136767
2,Lokomotiv Kuban,0.120512
3,Partizan,0.112220
4,Cedevita,0.098044
...,...,...
85,Fuenlabrada,0.000000
86,Dinamo Sassari,0.000000
87,Dijon,0.000000
88,Khimik,0.000000


In [67]:
ens_proba=(y_pre_proba_lgr[:,1]+y_pre_proba_rfc[:,1]+y_pre_proba_alg[:,1])/3


print("------Ensemble 3 classifier-------")
print()
print("Test BS:",round(brier_score_loss(y_test_ec,ens_proba),3))
print()
y_pre_ens=np.where(pd.Series(ens_proba)>0.5,1,0)
print("Test Accuracy:",round(accuracy_score(y_pre_ens,y_test_ec),3))
print()
print("Test F1-score:",round(f1_score(y_pre_ens,y_test_ec),3))


------Ensemble 3 classifier-------

Test BS: 0.189

Test Accuracy: 0.75

Test F1-score 0.818
